In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla P100-PCIE-16GB, 418.67, 16280 MiB


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 1.29 ms


In [ ]:
git clone 

In [18]:
# import os
# os.chdir("drive/My Drive/hip-mdp-public/")
% ls

acrobot_run.py            hiv_simulator/      results/
acrobot_simulator/        LICENSE.txt         run.py
BayesianNeuralNetwork.py  preset_parameters/  simple_grid_run.py
ExperienceReplay.py       PriorityQueue.py    Simple_HiPMDP.py
grid_run.py               __pycache__/        SimpleNetwork.py
grid_simulator/           Qnetwork.py         toy_example.ipynb
HiPMDP.py                 README.md
time: 349 ms


In [20]:
! git config --global user.email "lushao.ls1232@gmail.com"
! git config --global user.name "lushao1232"

time: 635 ms


In [23]:
! git add results/grid_exp_buffer
! git add results/grid_network_weights

time: 1.44 s


In [ ]:
! git commit -a -m "from colab2"

In [24]:

! git push

[master 073ec46] from colab2
 3 files changed, 1 insertion(+), 1 deletion(-)
 create mode 100644 results/grid_exp_buffer
 create mode 100644 results/grid_network_weights
 rewrite toy_example.ipynb (86%)
fatal: could not read Username for 'https://github.com': No such device or address
time: 2.87 s


In [ ]:
!pip install autograd==1.2
!pip install ipython-autotime

%load_ext autotime

  Created wheel for autograd: filename=autograd-1.2-cp36-none-any.whl size=44418 sha256=2b704c6381d10d17760c48017260b548c62acce5beb947c5730f08139975d603
  Stored in directory: /root/.cache/pip/wheels/72/6f/c2/40f130cca2c91f31d354bf72de282922479c09ce0b7853c4c5
Successfully built autograd
  Found existing installation: autograd 1.3
    Uninstalling autograd-1.3:
      Successfully uninstalled autograd-1.3
  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1831 sha256=9a9cdb7153cbd2db8597eb5442831201c22bd2c7be297d945b4cd04c66a979a1
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [ ]:
%matplotlib inline

import pickle, os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid",{'axes.grid' : False})

from BayesianNeuralNetwork import *
from HiPMDP import HiPMDP
from __future__ import print_function
from ExperienceReplay import ExperienceReplay
from multiprocessing import Pool

if not os.path.isdir('./results'):
    os.mkdir('results')

time: 4.96 s


## Generate batch of transition data
* Toy domain
* 2 different task instances
* Agent learning model-free using e-greedy policy based on DQN

In [ ]:
domain = 'grid'
run_type = 'modelfree'
num_batch_instances = 2
preset_hidden_params = [{'latent_code':1},{'latent_code':2}]
ddqn_learning_rate = 0.0005
episode_count = 500
bnn_hidden_layer_size = 25
bnn_num_hidden_layers = 3
bnn_network_weights = None
eps_min = 0.15
test_inst = None
create_exp_batch = True
state_diffs = True
grid_beta = 0.1
batch_generator_hipmdp = HiPMDP(domain,preset_hidden_params,
                                ddqn_learning_rate=ddqn_learning_rate,
                                episode_count=episode_count,
                                run_type=run_type, eps_min=eps_min,
                                create_exp_batch=create_exp_batch,
                                num_batch_instances=num_batch_instances,
                                grid_beta=grid_beta,
                                print_output=True)

time: 1.89 s


#### *The authors firmly suggest that those walking through this example for the first time test and evaluate the HiP-MDP with the "grid" domain as its runs quicker. The remainder of this notebook is therefore built to support investigation into this simple toy domain*

However, we provide example code of how to intialize a batch for use in either the Acrobot or HIV domains in the cell below. 

This example is meant to replace the code in lines 1-4 in the cell above. Testing the HiP-MDP entails selecting any additional parameter setting beyond the batch used here to train the initial BNN and latent weights.

Pay particular attention to other parameters used for training the BNN below. They are hard coded here for demonstration purposes. You may want to reference HiPMDP.py (included in this repo) for domain specific parameters

In [ ]:
## Example code of how to initialize a batch generator for HIV or Acrobot
# domain = 'hiv' # 'acrobot'
# with open('/'+domain+'_preset_hidden_params','r') as f:
#      = pickle.load(f)
    
# run_type = 'modelfree'
# num_batch_instances = 5
# preset_hidden_params = [:num_batch_instances]

This next cell will take a fair amount of time to run, it is generating several examples from the two different task instances to train the BNN and Q-network on.

In [ ]:
! git pull

In [ ]:
(exp_buffer, networkweights, rewards, avg_rwd_per_ep, full_task_weights,
     sys_param_set, mean_episode_errors, std_episode_errors) = batch_generator_hipmdp.run_experiment()

In [ ]:
with open('results/{}_exp_buffer'.format(domain),'wb') as f:
    pickle.dump(exp_buffer,f)

In [ ]:
with open('results/{}_exp_buffer'.format(domain),'rb') as f:
     exp_buffer = pickle.load(f)

time: 1.19 s


#### Reformat Data

In [ ]:
# Create numpy array 
exp_buffer_np = np.vstack(exp_buffer)
# Collect the instances that each transition came from
inst_indices = exp_buffer_np[:,4]
inst_indices = inst_indices.astype(int)
# Group experiences by instance
# Create dictionary where keys are instance indexes and values are np.arrays experiences
exp_dict = {}
for idx in range(batch_generator_hipmdp.instance_count):
    exp_dict[idx] = exp_buffer_np[inst_indices == idx]
X = np.array([np.hstack([exp_buffer_np[tt,0],exp_buffer_np[tt,1]]) for tt in range(exp_buffer_np.shape[0])])
y = np.array([exp_buffer_np[tt,3] for tt in range(exp_buffer_np.shape[0])])
num_dims = 2
num_actions = 4
num_wb = 5
if state_diffs:
    # subtract previous state
    y -= X[:,:num_dims]

time: 839 ms


## Train BNN and learn latent weights using batch data

#### Set up BNN and latent weights

In [ ]:
relu = lambda x: np.maximum(x, 0.)
param_set = {
    'bnn_layer_sizes': [num_dims+num_actions+num_wb]+[bnn_hidden_layer_size]*bnn_num_hidden_layers+[num_dims],
    'weight_count': num_wb,
    'num_state_dims': num_dims,
    'bnn_num_samples': 50,
    'bnn_batch_size': 32,
    'num_strata_samples': 5,
    'bnn_training_epochs': 1,
    'bnn_v_prior': 1.0,
    'bnn_learning_rate': 0.00005,
    'bnn_alpha':0.5,
    'wb_num_epochs':1,
    'wb_learning_rate':0.0005
}
# Initialize latent weights for each instance
full_task_weights = np.random.normal(0.,0.1,(batch_generator_hipmdp.instance_count,num_wb))
# Initialize BNN
network = BayesianNeuralNetwork(param_set, nonlinearity=relu)

time: 13 ms


In [ ]:
# Compute error before training
l2_errors = network.get_td_error(np.hstack((X,full_task_weights[inst_indices])), y, location=0.0, scale=1.0, by_dim=False)
print ("Before training: Mean Error: {}, Std Error: {}".format(np.mean(l2_errors),np.std(l2_errors)))
np.mean(l2_errors),np.std(l2_errors)
print ("L2 Difference in latent weights between instances: {}".format(np.sum((full_task_weights[0]-full_task_weights[1])**2)))

Before training: Mean Error: 0.2480777715058384, Std Error: 0.09922183607463823
L2 Difference in latent weights between instances: 0.058042493511295104
time: 1.68 s


#### Oscillate between training BNN and latent weights

In [ ]:
def get_random_sample(start,stop,size):
    indices_set = set()
    while len(indices_set) < size:
        indices_set.add(np.random.randint(start,stop))
    return np.array(list(indices_set))

time: 3.15 ms


We slowly train the BNN and latent weights $w_b$ so as to avoid prematurely fitting to the separate modes of input. We found that without taking this approach, the BNN associated the latent parameters as noise and ignored their contribution to the performance of the predictive model.

In [ ]:
# size of sample to compute error on
sample_size = 10000
for i in range(40):    
    # Update BNN network weights
    network.fit_network(exp_buffer_np, full_task_weights, 0, state_diffs=state_diffs,
                        use_all_exp=True)
    print('finished BNN update '+str(i))
    if i % 4 == 0:
        #get random sample of indices
        sample_indices = get_random_sample(0,X.shape[0],sample_size)
        l2_errors = network.get_td_error(np.hstack((X[sample_indices],full_task_weights[inst_indices[sample_indices]])), y[sample_indices], location=0.0, scale=1.0, by_dim=False)
        print ("After BNN update: iter: {}, Mean Error: {}, Std Error: {}".format(i,np.mean(l2_errors),np.std(l2_errors)))
    # Update latent weights
    for inst in np.random.permutation(batch_generator_hipmdp.instance_count):
        full_task_weights[inst,:] = network.optimize_latent_weighting_stochastic(
            exp_dict[inst],np.atleast_2d(full_task_weights[inst,:]),0,state_diffs=state_diffs,use_all_exp=True)
    print ('finished wb update '+str(i))
    # Compute error on sample of transitions
    if i % 4 == 0:
        #get random sample of indices
        sample_indices = get_random_sample(0,X.shape[0],sample_size)
        l2_errors = network.get_td_error(np.hstack((X[sample_indices],full_task_weights[inst_indices[sample_indices]])), y[sample_indices], location=0.0, scale=1.0, by_dim=False)
        print ("After Latent update: iter: {}, Mean Error: {}, Std Error: {}".format(i,np.mean(l2_errors),np.std(l2_errors)))
        # We check to see if the latent updates are sufficiently different so as to avoid fitting [erroneously] to the same dynamics
        print ("L2 Difference in latent weights between instances: {}".format(np.sum((full_task_weights[0]-full_task_weights[1])**2)))

In [ ]:
network_weights = network.weights

In [ ]:
with open('results/{}_network_weights'.format(domain), 'wb') as f:
    pickle.dump(network.weights, f)

In [ ]:
with open('results/{}_network_weights'.format(domain), 'rb') as f:
    network_weights = pickle.load(f)

time: 597 ms


## Learn dynamics and policy for new instance using HiP-MDP with embedded latent weights

In [ ]:
results = {}
run_type = 'full'
create_exp_batch = False
# episode_count = 20
episode_count = 10 # reduce episode count for demonstration since HiPMDP learns policy quickly
for run in range(5):
    for test_inst in [0,1]:
        test_hipmdp = HiPMDP(domain,preset_hidden_params, 
                         ddqn_learning_rate=ddqn_learning_rate, 
                         episode_count=episode_count,
                         run_type=run_type,
                         bnn_hidden_layer_size=bnn_hidden_layer_size,
                         bnn_num_hidden_layers=bnn_num_hidden_layers,
                         bnn_network_weights=network_weights, test_inst=test_inst,
                         eps_min=eps_min, create_exp_batch=create_exp_batch,grid_beta=grid_beta,print_output=True)
        results[(test_inst,run)] = test_hipmdp.run_experiment()
        with open('results/{}_results'.format(domain),'wb') as f:
            pickle.dump(results,f)

In [ ]:
with open('results/{}_results'.format(domain),'rb') as f:
    results = pickle.load(f)

In [ ]:
# Group rewards, errors by instance
reward_key = 'Reward'
error_key = 'BNN Error'
clean_results = {(0,reward_key):[],(0,error_key):[],(1,reward_key):[],(1,error_key):[]}
for test_inst in [0,1]:
    for run in range(5):
        clean_results[(test_inst,reward_key)].append(results[(test_inst,run)][2])
        clean_results[(test_inst,error_key)].append(results[(test_inst,run)][6])
    clean_results[(test_inst,reward_key)] = np.vstack(clean_results[(test_inst,reward_key)])
    clean_results[(test_inst,error_key)] = np.vstack(clean_results[(test_inst,error_key)])


In [ ]:
def plot_results(clean_results, test_inst):
    f, ax_array = plt.subplots(2,figsize=(7,7))
    result_names = [reward_key,error_key]
    for result_idx in range(2):
        result = result_names[result_idx]
        mean_result = np.mean(clean_results[(test_inst,result)], axis=0)
        std_result = np.std(clean_results[(test_inst,result)], axis=0)
        ax_array[result_idx].errorbar(x=np.arange(len(mean_result)),y=mean_result,yerr=std_result)
        _ = ax_array[result_idx].set_ylim((np.min(mean_result)-0.01,np.max(mean_result)+0.01))
        ax_array[result_idx].set_ylabel(result)
    ax_array[1].set_xlabel("Episode")
    f.suptitle("Full HiP-MDP Training Results Instance {}".format(test_inst),fontsize=12)
    plt.savefig('results/{}_results_{}.png'.format(domain, test_inst))

time: 6 ms


In [ ]:
plot_results(clean_results, 0)

In [ ]:
plot_results(clean_results, 1)